In [1]:
import os
import wikipediaapi
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, ChatMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import BaseTool, StructuredTool, tool
from serpapi import GoogleSearch
from pydantic import BaseModel, Field
from dotenv import load_dotenv
_ = load_dotenv()
serp_api_key = os.environ.get('SERP_API_KEY')

##### Create Agent using Langgraph

In [2]:
from langgraph.checkpoint.memory import MemorySaver  
from langgraph.prebuilt import create_react_agent

##### Tools

In [3]:
@tool
def search_wikipedia(query: str):
    """Search wikipedia for factual information."""
    wiki_api=wikipediaapi.Wikipedia("Info Agent","en")
    page = wiki_api.page(query)
    if page.exists():
        return  page.summary[:500]  
    return 'No relevant information found.'

@tool
def search_google(query: str):
    """Google Search: Search google for latest updates and news."""
    search = GoogleSearch({"q": query, "api_key": serp_api_key})
    results = search.get_dict()
    if 'organic_results' in results:
        top_result = results['organic_results'][0]
        title = top_result.get("title")
        link = top_result.get("link")
        snippet = top_result.get("snippet")
        return  snippet
    return 'No relevant information found.'

##### Create Agent using langgraph prebuild react agent

In [4]:
memory = MemorySaver()
model = ChatOpenAI(temperature=0)
tools = [search_google, search_wikipedia]
system_message="You are helpful but sassy assistant"

In [5]:
langgraph_agent_executor = create_react_agent(
    model, tools, state_modifier=system_message, checkpointer=memory
)

In [8]:
config = {"configurable": {"thread_id": "test-thread"}}
results=langgraph_agent_executor.invoke({"messages": [("user", "What is covid 19")]},
    config
)["messages"][-1].content

In [9]:
results

'COVID-19, also known as Coronavirus disease 2019, is a contagious disease caused by the coronavirus SARS-CoV-2. The first known case was identified in Wuhan, China, in December 2019. It is believed that the virus entered human populations through natural zoonosis. Various social and environmental factors have contributed to the spread of the virus.'